17-数组检索子程序之三
====
Inserting data into arrays
----

| Inserting   data into arrays               |                                                              |
| ------------------------------------------ | ------------------------------------------------------------ |
| place(arr, mask, vals)                     | Change elements   of an array based on conditional and input values. |
| put(a, ind, v[, mode])                     | Replaces   specified elements of an array with given values. |
| put_along_axis(arr, indices, values, axis) | Put   values into the destination array by matching 1d index and data slices. |
| putmask(a, mask, values)                   | Changes   elements of an array based on conditional and input values. |
| fill_diagonal(a, val[, wrap])              | Fill   the main diagonal of the given array of any dimensionality. |

# numpy.place
`numpy.place(arr, mask, vals)`

根据条件和输入值更改数组的元素。

与`np.copyto（arr，vals，where = mask）`类似，区别在于该位置使用vals的前N个元素，其中N是掩码中True值的数量，而copyto使用mask为True的元素。

请注意，`extract`提取的元素与`place`完全相反。

## Parameters:	
**arr** : ndarray，将数据放入的数组。

**mask** : array_like，布尔掩码数组。必须与 a 有相同的大小。

**vals** : 1-D sequence

放入 a 的值。 仅使用前N个元素，其中N是掩码中的True值的数量。 如果vals小于N，则重复执行，如果要屏蔽 a 的元素，则此序列必须为非空。

## 示例

In [2]:
import numpy as np
>>> arr = np.arange(6).reshape(2, 3)
>>> np.place(arr, arr>2, [44, 55])
>>> arr

array([[ 0,  1,  2],
       [44, 55, 44]])

# numpy.put
`numpy.put(a, ind, v, mode='raise')`

用给定值替换数组的指定元素。

索引适用于展平的目标数组。 put大致相当于：`a.flat[ind] = v`

## Parameters:	
**a** : ndarray，目标数组。

**ind** : array_like，目标索引，解释为整数。

**v** : array_like，放置在目标索引处的值。 如果v比ind短，则根据需要重复。

**mode** : `{‘raise’, ‘wrap’, ‘clip’}`, 可选参数。指定超出边界的索引的行为。

- ‘raise’ – raise an error (default)
- ‘wrap’ – wrap around
- ‘clip’ – clip to the range

'clip'模式意味着所有过大的索引都被索引沿着该轴的最后一个元素所取代。 请注意，这会禁用带负数的索引。

## 示例

In [3]:
>>> a = np.arange(5)
>>> np.put(a, [0, 2], [-44, -55])
>>> a

array([-44,   1, -55,   3,   4])

In [4]:
>>> a = np.arange(5)
>>> np.put(a, 22, -5, mode='clip')
>>> a

array([ 0,  1,  2,  3, -5])

## numpy.put_along_axis
`numpy.put_along_axis(arr, indices, values, axis)`

通过匹配1d索引和数据切片将值放入目标数组。

这将遍历1d切片，指定沿索引和数据数组中指定轴，并使用前者将值放入后者中。 这些切片可以是不同的长度。

沿轴返回索引的函数（如argsort和argpartition）会为此函数生成合适的索引。

## Parameters:	
**arr**: ndarray (Ni…, M, Nk…)，目标数组。

**indices**: ndarray (Ni…, J, Nk…)

索引沿着arr的每个1d切片变化。必须与arr的维度匹配，但是Ni和Nj中的维度可以是1，以对arr广播。

**values**: array_like (Ni…, J, Nk…)。

在这些索引中插入的值。它的形状和大小被广播以与索引相匹配。

**axis**: int

1d切片沿着该轴。如果axis为None，则将目标数组视为已创建平展的1d视图。

**注意**
这相当于（但更快）以下使用`ndindex`和`s_`，它们将`ii`和`kk`中的每一个设置为索引的元组：

```
Ni, M, Nk = a.shape[:axis], a.shape[axis], a.shape[axis+1:]
J = indices.shape[axis]  # Need not equal M

for ii in ndindex(Ni):
    for kk in ndindex(Nk):
        a_1d       = a      [ii + s_[:,] + kk]
        indices_1d = indices[ii + s_[:,] + kk]
        values_1d  = values [ii + s_[:,] + kk]
        for j in range(J):
            a_1d[indices_1d[j]] = values_1d[j]
```

同样地，消除内部循环，最后两行是：

`a_1d[indices_1d] = values_1d`

## 示例

In [5]:
# 对下面的示例数组
>>> a = np.array([[10, 30, 20], [60, 40, 50]])

In [10]:
#我们可以用以下代码替换最大值：
>>> ai = np.expand_dims(np.argmax(a, axis=1), axis=1)
>>> ai

array([[1],
       [0]], dtype=int64)

In [11]:
# >>> np.put_along_axis(a, ai, 99, axis=1) # 在1.15.0版中才有。
>>> a

array([[10, 30, 20],
       [60, 40, 50]])

# numpy.putmask
`numpy.putmask(a, mask, values)`

根据条件和输入值改变阵列的元素。

为每个n设置`a.flat [n] = values [n]`，其中`mask.flat [n] == True`。

如果值与`a`和`mask`的大小不同，那么它将重复。 这给出了与`[mask] = values`不同的行为。

## Parameters:	
**a** : array_like，目标数组。

**ask** : array_like，布尔掩码数组。它必须与a有相同的shape。

**values** : array_like，将值放入一个掩码中是正确的。如果值小于a，那么它就会重复。

## 示例

In [12]:
>>> x = np.arange(6).reshape(2, 3)
>>> np.putmask(x, x>2, x**2)
>>> x

array([[ 0,  1,  2],
       [ 9, 16, 25]])

如果值比 a 小，它就被重复。

In [13]:
>>> x = np.arange(5)
>>> np.putmask(x, x>1, [-33, -44])
>>> x

array([  0,   1, -33, -44, -33])

# numpy.fill_diagonal
`numpy.fill_diagonal(a, val, wrap=False)`

填充任何维度的给定数组的主对角线。

对于`a.ndim> = 2`的数组`a`，对角线是索引`a [i，...，i]`全部相同的位置列表。 此函数就地修改输入数组，它不返回值。

## Parameters:	
**a** : array, at least 2-D.要填充对角线的数组，就地进行修改。

**val** : scalar, 要写在对角线上的值，其类型必须与数组a的类型兼容。

**wrap** : bool

对于NumPy版本高达`1.6.2`的高矩阵，对角线“包裹”在N列之后。 您可以使用此选项具有此行为。 这只影响高矩阵。

**注意**

版本1.4.0中的新功能。

这个功能可以通过diag_indices获得，但在内部这个版本使用了更快的实现，它从不构造索引并使用简单的切片。

## 示例

In [14]:
>>> a = np.zeros((3, 3), int)
>>> np.fill_diagonal(a, 5)
>>> a

array([[5, 0, 0],
       [0, 5, 0],
       [0, 0, 5]])

同样的函数可以在4-D数组上操作：

In [15]:
>>> a = np.zeros((3, 3, 3, 3), int)
>>> np.fill_diagonal(a, 4)

为清晰起见，我们只显示了几个块：

In [16]:
a[0,0]

array([[4, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [17]:
a[1,1]

array([[0, 0, 0],
       [0, 4, 0],
       [0, 0, 0]])

In [18]:
a[2,2]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 4]])

wrap选项只影响高矩阵：

In [21]:
>>> # tall matrices no wrap
>>> a = np.zeros((5, 3),int)
>>> np.fill_diagonal(a, 4)
>>> a

array([[4, 0, 0],
       [0, 4, 0],
       [0, 0, 4],
       [0, 0, 0],
       [0, 0, 0]])

In [22]:
>>> # tall matrices wrap
>>> a = np.zeros((5, 3),int)
>>> np.fill_diagonal(a, 4, wrap=True)
>>> a

array([[4, 0, 0],
       [0, 4, 0],
       [0, 0, 4],
       [0, 0, 0],
       [4, 0, 0]])

In [23]:
>>> # wide matrices
>>> a = np.zeros((3, 5),int)
>>> np.fill_diagonal(a, 4, wrap=True)
>>> a

array([[4, 0, 0, 0, 0],
       [0, 4, 0, 0, 0],
       [0, 0, 4, 0, 0]])